<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Heart Failure prediction using teradataml
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p> 
    <center><img src="images/heart.webp" width="350"/></center>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Source: <a href = 'https://www.healthline.com/health/heart-disease/statistics#How-much-does-it-cost?'>Healthline</a></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>According to the CDC, the number of emergency room visits in 2017 for issues related to the heart and blood vessels was nearly 5 million. Plus, heart attacks are one of the most expensive conditions treated in U.S. hospitals - cardiac care costs an estimated $11.5 billion a year. Being able to identify these conditions in people before a heart attack can not only help patients save money, but also save their lives. With Teradata Vantage and ClearScape Analytics, users can better understand the full picture of their patient’s health. Doctors and physicians can have a better understanding of their health and predict whether heart failure is expected.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>It's estimated that by 2035, more than 45 percent of Americans are projected to have some form of cardiovascular disease, meaning that heart failure prediction is an issue that will need to be better predicated. With Teradata’s capabilities, machine learning can be useful in heart failure prediction as it can analyze large amounts of data from multiple sources and identify complex patterns that may be difficult for humans to recognize. This can potentially improve the accuracy of prediction models and help healthcare professionals identify patients who are at high risk for heart failure, allowing for earlier intervention and better outcomes.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Comprehensive health predictions and a reduced number of false positive and false negative results.</li>
    <li>Reduced cost to patients and hospitals caused by heart attacks.</li>
    <li>Identify patterns and symptoms leading to heart failure to ensure early intervention.</li>
    <li>Advanced research and development stemming from the results of the data and models produced.</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Machine Learning and AI have a proven track record of improving patient outcomes and well-being across the entire healthcare industry. Traditional approaches to data preparation, model development, and deployment rely on manual, error-prone processes that prevent enterprises from realizing the true value of these tools and techniques.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>However, Vantage provides these same proven data preparation and machine learning capabilities, integrated as native ClearScape Analytic functions.  This allows organizations to drastically reduce data preparation, model development, and testing time, while allowing for much more frequent and iterative testing and tuning to ensure maximum life-critical accuracy.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Furthermore, the exact same development pipeline can be deployed seamlessly to production, eliminating the traditional development-to-deployment gap in the ML and AI industry.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Data:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    This is a simulated dataset based on real hospital administrative data for England called Hospital Episodes Statistics. Every public (National Health Service, NHS) hospital in the country must submit records for every admission; private hospitals also submit records for any NHS patients that they treat.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# system packages
import sys
import warnings
warnings.filterwarnings("ignore")

from teradataml import *
from teradataml import valib

# Dataset packages 
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from sklearn.metrics import ConfusionMatrixDisplay

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "whitegrid")

%matplotlib inline
configure.val_install_location = "val"
display.max_rows = 5

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Heart_Failure_Prediction_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_HeartFailure_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_HeartFailure_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 5 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>

In [ ]:
heart_failure = DataFrame(in_schema('DEMO_HeartFailure', 'heart_failure'))

In [ ]:
print(heart_failure.shape)
heart_failure

<i><b>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</b></i>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the dataset above, we have a total of 31 columns, and the 'death' column is our predicted column where 1 means the patient died, and 0 means they did not.
<br><br>    
Now, let's check the data for people who died and those who did not.
</p>

In [ ]:
# Sample data for people who did not die
print("No. of people who did not die:", heart_failure[heart_failure.death == 0].shape[0])
heart_failure[heart_failure.death == 0]

In [ ]:
# Sample data for people who did die
print("No. of people who did die:", heart_failure[heart_failure.death == 1].shape[0])
heart_failure[heart_failure.death == 1]

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 Mortality Rate by Gender</b></p>

In [ ]:
grp_gen = heart_failure.select(['gender','death']).groupby(['gender']).agg(['mean', 'count']).to_pandas()
sns.barplot(x = 'gender', y = 'mean_death', data = grp_gen)
plt.xticks(ticks = [0, 1], labels = ['male', 'female'])
plt.title('Mortality rate by gender')
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Based on the graph we see above, we conclude that gender does not appear to be a determining factor in mortality rates related to heart failure.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage's Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI, or many Python modules available such as plotly, seaborn, etc. We can perform all the calculations and pre-processing in Vantage, passing only the necessary information to visualization tools. This approach not only makes the calculations faster but also reduces the overall time due to less data movement between tools.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.2 Mortality Rate by Age</b></p>

In [ ]:
grp_gen = heart_failure.select(['age','death']).groupby(['age']).agg(['mean', 'count'])

figure = Figure(width=1500, height=600, heading="Mortality rate by age")

plot = grp_gen.plot(
    x=grp_gen.age,
    y=grp_gen.mean_death,
    kind='bar',
    figure=figure
)

plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can observe a noticeable pattern from the graph, revealing a positive correlation between age and mortality rates associated with heart failure.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.3 Correlation Matrix</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The corr() function returns the Sample Pearson product moment correlation coefficient for all non-null data point pairs of ColumnExpression.</p>
<i><b>*The following cell might take upto 2 mins to run</b></i>

In [ ]:
# Create an empty correlation matrix
correlation_matrix = pd.DataFrame(index=heart_failure.columns[1:], columns=heart_failure.columns[1:])

for col1, col2 in combinations(heart_failure.columns[1:], 2):
    correlation_value = heart_failure.assign(drop_columns = True, _corr = heart_failure[col1].corr(heart_failure[col2])).get_values()[0][0]

    # Fill in the correlation matrix for both col1 -> col2 and col2 -> col1
    correlation_matrix.loc[col1, col2] = correlation_value
    correlation_matrix.loc[col2, col1] = correlation_value

# Fill in the diagonal with 1 since it's the correlation of a column with itself
correlation_matrix = correlation_matrix.fillna(1.0)

# Convert all columns to float data type
correlation_matrix = correlation_matrix.astype(float)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI, or many Python modules available such as plotly, seaborn, etc. We can perform all the calculations and pre-processing on Vantage, passing only the necessary information to visualization tools. This approach not only makes the calculations faster but also reduces the time due to less data movement between tools.</p>

In [ ]:
# Set figure size to 20 inches by 8 inches
sns.set(rc = {"figure.figsize": (20, 8)})
# Create a heatmap to visualize the correlation matrix
ax = sns.heatmap(correlation_matrix, annot = False, cmap = 'coolwarm', vmin = -1, vmax = 1)

# Set title and show plot
plt.title('Multivariate Correlation Matrix')
plt.show()

<p style='font-size:16px;font-family:Arial;color:#00233C'>Few observations from the correlation matrix above are:</p>
<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>We observe a positive correlation between Age and mortality.</li>
    <li>We find a positive correlation between Cancer and metastatic cancer.</li>
    <li>We notice a positive correlation between the number of prior appointments attended and the number of appointments missed in the previous year.</li>
    <li>We see a negative correlation between our age and follow-up time, indicating that as we age, follow-up time tends to decrease, leading to more frequent hospital visits.</li>
    <li>The negative correlation between our age and the number of prior appointments missed suggests that as we get older, we tend to miss fewer appointments, reflecting an increased seriousness about our health.</li>
</ul>
<p style='font-size:16px;font-family:Arial;color:#00233C'>While these correlations exist, we acknowledge that they may not be strong enough to justify removing any columns from our dataset.</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.4 Distribution plots for numeric variables</b></p>

In [ ]:
from tqdm import tqdm

cols = ["los","age","prior_appts_attended","prior_dnas","fu_time"]

fig, axes = subplots(nrows=2, ncols=3)

# Plot the data
for i in tqdm(range(len(cols))):
    obj = Histogram(
            data=heart_failure,
            target_columns=cols[i],
            method_type="STURGES"
    ).result

    plot = obj.plot(
        x=obj.MinValue,
        y=obj.CountOfValues,
        kind='bar',
        figure=fig,
        grid_linestyle='--',
        grid_linewidth=0.5,
        ylabel='Frequency',
        legend=f'Distribuion of {cols[i]}',
        ax=axes[i]
    )

plot.show()

<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li>We observe that the distribution of length of stay (LOS) indicates that the majority of hospitalizations are less than 20 days.</li>
    <li>We notice that the distribution of age suggests that individuals aged 60 and above are more likely to be hospitalized.</li>
    <li>We can see that the distribution of prior appointments attended suggests that most people attend fewer than 10 appointments.</li>
    <li>We find that the distribution of prior missed appointments (prior DNAs) indicates that the majority of people do not miss appointments.</li>
    <li>We see that the distribution of follow-up time suggests that a large number of individuals have follow-up time less than 100 days.</li>
</ul>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Data Preparation</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We classify the columns into an ID column, numerical, categorical, and binary ones.</p>

In [ ]:
num_x = ["los","age","prior_appts_attended","prior_dnas","fu_time"]
cat_x = ["gender", "quintile","ethnicgroup"]
bin_x = ["death", "cancer", "cabg","crt", "defib","dementia", "diabetes","hypertension", "ihd",
         "mental_health", "arrhythmias", "copd", "obesity","pvd", "renal_disease", "valvular_disease",
         "metastatic_cancer", "pacemaker", "pneumonia", "pci", "stroke", "senile"]
idcol = ["id"]

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find one-hot encoding useful when we need to re-express a categorical data element as one or more numeric data elements. This involves creating a binary numeric field for each categorical data value.</p>

In [ ]:
# 1 - male, 2 - female
values1 = {1: "Gender"}
dummy1 = OneHotEncoder(values = values1, columns = "gender")

# quintile (socio-economic status for patient's neighbourhood, from 1 (most affluent) to 5 (poorest))
values2 = {1: "q_richest", 2: "q_rich", 3: "q_average", 4: "q_poor", 5: "q_poorest"}
dummy2 = OneHotEncoder(values = values2, columns = "quintile")

# 1 - White, 2 - Black, 3 - Indian Subcontinent, 8 - Not Known, 9 - Other
values3 = {1: "White", 2: "Black", 3: "Indian_Subcontinent", 8: "Not_Known", 9:"Other"}
dummy3 = OneHotEncoder(values = values3, columns = "ethnicgroup")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We use FillNa to perform missing value/null replacement transformations. We apply Z-Score to transform each column value into the number of standard deviations from the mean value of the column.</p>

In [ ]:
fn = FillNa(
    style = "mode",
    columns = num_x
)
zs = ZScore(
    columns = num_x,
    out_columns = num_x, 
    fillna = fn
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We keep the other variables that do not need transformation.</p>

In [ ]:
retain = Retain(columns = bin_x)

<p style='font-size:16px;font-family:Arial;color:#00233C'>ClearScape Analytics provides Data Scientists and other users with over 50 advanced analytic functions built directly in the Advanced SQL Engine, which is a core capability of Teradata Vantage. These functions support the entire data science process, including exploratory data analysis, data preparation and feature engineering, hypothesis testing, as well as statistical and machine learning model building and scoring.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Transform function reads a teradataml DataFrame and produces an output containing transformed columns.</p>

In [ ]:
# Process the transformation
df_transformed = valib.Transform(
    data = heart_failure, 
    zscore = zs, 
    one_hot_encode = [dummy1, dummy2, dummy3],
    retain = retain,
    index_columns = idcol,
    key_columns = idcol
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We save the transformed dataframe into a table <b>heart_failure_clean</b>.</p>

In [ ]:
df_transformed.result.to_sql(
    "heart_failure_clean",
    primary_index = "id",
    if_exists = "replace"
)

df_transformed.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We split the data in training and testing datasets in 75:25 ratio.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
    data = DataFrame('heart_failure_clean'),
    id_column = "id",
    train_size = 0.75,
    test_size = 0.25,
    seed = 42
)

df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

copy_to_sql(df_train, table_name = 'heart_failure_train', if_exists = 'replace')
copy_to_sql(df_test, table_name = 'heart_failure_test', if_exists = 'replace')

In [ ]:
print("Training Set = " + str(df_train.shape[0]) + " | Testing Set = " + str(df_test.shape[0]))

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Model Training</b>

<p style='font-size:16px;font-family:Arial;color:#00233C'>The TD_GLM function as a generalized linear model (GLM) for performing regression and classification analyses on data sets, where the response follows an exponential or binomial family distribution.
    <br>
    <br>
    When using the function, we should provide only numeric features. We need to convert the categorical features to numeric values before training. Additionally, during training, TD_GLM function skips the rows with missing (null) values.
</p>

In [ ]:
from teradataml import GLM, TDGLMPredict

glm_model = GLM(
    data = df_train,
    input_columns = '2:38', 
    response_column = 'death',
    family = 'Binomial',
    batch_size = 50,
    iter_max = 100,
    lambda1 = 0.02,
    alpha = 0.05,
    iter_num_no_change = 5,
    intercept = True,
    learning_rate = 'optimal',
    initial_eta = 0.05,
    local_sgd_iterations = 20
)

In [ ]:
glm_model.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage's Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI, or many Python modules available such as plotly, seaborn, etc. We can perform all the calculations and pre-processing on Vantage, passing only the necessary information to visualization tools. This approach not only makes the calculations faster but also reduces the time due to less data movement between tools.</p>

In [ ]:
feat_imp = glm_model.result[glm_model.result['attribute'] > 0].sort('estimate', ascending = False).to_pandas()

# Specify figure size
fig, ax = plt.subplots(figsize = (10, 8))

# Use ax.barh() for horizontal bar chart
ax.barh(feat_imp['predictor'], feat_imp['estimate'], edgecolor = 'red')

# Add text labels on right of the bars
for x, y in zip(feat_imp['estimate'], feat_imp['predictor']):
    ax.text(x, y, str(round(x, 2)), ha = 'left', va = 'center')

# Set y-axis label
ax.set_xlabel('Estimate')

plt.title('Feature importance')

plt.show()

from IPython.display import display, HTML

feat1, feat2, feat3, feat4 = (
    list(feat_imp.iloc[:2].predictor)[0],
    list(feat_imp.iloc[:2].predictor)[1],
    list(feat_imp.iloc[-2:].predictor)[0],
    list(feat_imp.iloc[-2:].predictor)[1],
)

html_content = f"""
<p style='font-size:16px;font-family:Arial'>
The feature importances we displayed above indicate that <b>{feat1}, {feat2}, {feat3}, and {feat4}</b> are significant factors in predicting heart failure deaths.
<br>
Note that the top ones (negative coefficients) negatively affect the target variable, and that the bottom ones (positive coefficients) positively affect the target variable.
</p>
"""

display(HTML(html_content))

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Model Validation</b>
<p style='font-size:16px;font-family:Arial;color:#00233C'>
We, using the TD_GLMPredict function, predict target values (regression) and class labels (classification) for test data using a GLM model trained by the GLM function.
<br>
<br>
Similar to GLM, we standardize input features, such as using ScaleFit and ScaleTransform, before using them in the function. The function takes only numeric features. We must convert the categorical features to numeric values before prediction.
</p>

In [ ]:
glm_prediction = glm_model.predict(
    newdata = df_test,
    id_column = 'id',
    accumulate = 'death',
    output_prob = True,
    output_responses = ['0', '1'],
    family = 'Binomial'
)

out = glm_prediction.result.assign(prediction = glm_prediction.result.prediction.cast(type_ = BYTEINT))
out = out.assign(prediction = out.prediction.cast(type_ = VARCHAR(2)))
out = out.assign(death = out.death.cast(type_ = VARCHAR(2)))

In [ ]:
out

<p style='font-size:16px;font-family:Arial;color:#00233C'>In the above result, we consider the column <b>death</b> as our ground truth, <b>prediction</b> as our predicted output, and <b>(prob_0, prob_1)</b> as the probabilities of the output class.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>ClassificationEvaluator() function evaluates and emits various metrics of classification model based on its predictions on the data. Apart from accuracy, the secondary output data returns micro, macro, and weighted-averaged metrics of precision, recall, and F1-score values.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>This is a powerful function, and doesn't move data outside Vantage.</p>

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(
    data = out,
    observation_column = 'death',
    prediction_column = 'prediction',
    labels = ['0', '1']
)

In [ ]:
ClassificationEvaluator_obj.output_data.head(10)

<p style='font-size:16px;font-family:Arial;color:#00233C'>We consider recall more important when <b>Overlooked Cases (False Negatives)</b> are more costly than <b>False Alarms (False Positives)</b>. Because not “capturing” even one case of heart failure could result in death, our models should place emphasis on the recall score. It is far preferable for us to not “miss” anyone with heart failure, even if that means “flagging” some patients as having heart failure who actually do not have the disease.
</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Let's consider one example. Here we check for patient number 856.</p>

In [ ]:
heart_failure[heart_failure['id'] == 856]

<p style='font-size:16px;font-family:Arial;color:#00233C'>Based on the data provided, the individual in question has hypertension, arrhythmia, and an age of 98. We have identified these features as significant predictors of mortality by our model. Therefore, there was a high probability of death for this patient, which is further supported by the outcome indicating death.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Visualize the results</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI, or many Python modules available such as plotly, seaborn, etc. We can perform all the calculations and pre-processing on Vantage, passing only the necessary information to visualization tools. This approach not only makes the calculations faster but also reduces the time due to less data movement between tools.</p>

In [ ]:
# Compute confusion matrix
cm = ClassificationEvaluator_obj.result.get(['CLASS_1', 'CLASS_2']).get_values().T

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['No Death', 'Death'])
fig, ax = plt.subplots(figsize = (8, 8))
disp.plot(ax = ax, cmap = 'Blues', colorbar = True)

# Add labels and annotations
plt.title('GLM Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(ticks = [0, 1], labels = ['No Death', 'Death'])
plt.yticks(ticks = [0, 1], labels = ['No Death', 'Death'])

# Add text to the plot to show the actual values of the confusion matrix
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, f'{cm[i, j]}', ha = 'center', va = 'center', color = 'white' if cm[i, j] > cm.max()/1.4 else 'black')

# Remove grid lines
ax.grid(False)

# Show the plot
plt.show()

print(f'''
This means that out of all the actual non-death cases ({cm[0][0] + cm[0][1]}),
{round(cm[0][0]/(cm[0][0] + cm[0][1])*100, 2)}% were correctly classified as non-death, while
{round(cm[0][1]/(cm[0][0] + cm[0][1])*100, 2)}% were incorrectly classified as death.
Similarly, out of all the actual death cases ({cm[1][0] + cm[1][1]}),
{round(cm[1][1]/(cm[1][0] + cm[1][1])*100, 2)}% were correctly classified as death, while
{round(cm[1][0]/(cm[1][0] + cm[1][1])*100, 2)}% were incorrectly classified as non-death.
''')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demonstration, we have illustrated a simplified - but complete - overview of how we can implement a typical machine learning workflow completely inside the database using Vantage. This allows us to leverage Vantage's operational scale, power, and stability.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
tables = ['heart_failure_clean', 'heart_failure_train', 'heart_failure_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_HeartFailure');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Industry:</b> Healthcare</li>
    <li><b>Functionality:</b> Machine Learning</li>
    <li><b>Use Case:</b> Prediction Analysis</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li>
    <li><a href='https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a></li>
    <li><a href='https://www.teradata.com/Blogs/Forecasting-COVID-19-Using-Teradata-Vantage'>Forecasting COVID-19 Using Teradata Vantage</a></li>
</ul>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>

- `id`: patient id
- `death`: If the patient is deceased(boolean)
- `los`: hospital length of stay (in nights)
- `age`: age of the patient (in years)
- `gender`: gender of the patient (1-male, 2-female)
- `cancer`: If the patient has cancer (boolean)
- `cabg`: If the patient has gone through previous heart bypass i.e. Coronary Artery Bypass Graft procedure (boolean)
- `crt`: If the patient has gone through Cardiac Resynchronization Therapy i.e. a device - a treatment for heart failure(boolean)
- `defib`: If the patient has defibrillator implanted (boolean)
- `dementia`: If the patient has dementia (boolean)
- `diabetes`: If the patient has diabetes (boolean)
- `hypertension`: If the patient has hypertension (boolean)
- `ihd`: If the patient has Ischemic Heart Disease (boolean)
- `mental_health`: If the patient has been diagnosed with mental health issues (boolean)
- `arrhythmias`: If the patient has arrhythmia (boolean)
- `copd`: If the patient has Chronic Obstructive Pulmonary Disease (boolean)
- `obesity`: If the patient has obesity (boolean)
- `pvd`: If the patient has Peripheral Vascular Disease (boolean)
- `renal_disease`: If the patient has Renal Disease (boolean)
- `valvular_disease`: If the patient has Valvular Disease (boolean)
- `metastatic_cancer`: If the patient has Metastatic Cancer (boolean)
- `pacemaker`: If the patient has pacemaker (boolean)
- `pneumonia`: If the patient has pneumonia (boolean)
- `prior_appts_attended`: Number of outpatient appointments attended in the previous year
- `prior_dnas`: Number of outpatient appointments missed in the previous year
- `pci`: If the patient has gone though Percutaneous Coronary Intervention procedure (boolean)
- `stroke`: History of stroke
- `senile`: If the patient has Senile amyloidosis (SSA) (boolean)
- `quintile`: Socio-economic status for patient's neighborhood, from 1 (most affluent) to 5 (poorest)
- `ethnicgroup`: 1 - White, 2 - Black, 3 - Indian Subcontinent, 8 - Not Known, 9 - Other 
- `fu_time`: Follow-up time, i.e. time in days since admission to hospital

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>